字符串匹配，主串为main_str，模式串为pattern，查找的过程就是不断移动pattern的过程。如何高效的移动就是算法的意义所在。

***BF算法***

In [1]:
# 暴力匹配法
def search(main_str, pattern):
    m = len(main_str)
    p = len(pattern)
    if m == 0 or p == 0 or m < p:
        return -1
    for i in range(0, m-p+1):
        subStr = main_str[i:i+p]
        if subStr == pattern:
            return i
    return -1
main_str = 'acbcabdab'
pattern = 'dab'
search(main_str, pattern)

6

***Boyer-Moore 算法***  
基于*坏字符规则 (bad-character shift) 和 好后缀规则 (good-suffix shift)* ，尽可能排除无法匹配的位置，加快搜索的步骤。  


In [2]:
# ord 函数以一个字符（长度为1的字符串）作为参数，返回对应的 ASCII 数值
print(ord('A'))
print(ord('Z'))
print(ord('a'))
print(ord('z'))

65
90
97
122


**散列表**  
理想的散列表（哈希表）是包含「关键字」的具有固定大小的数组，能以常数时间执行「插入、删除、查找」。
每个关键字被映射到 0 到 N-1 范围内，并放到合适的位置，这个映射规则就称为散列函数。  
理想情况下，两个不同的关键字映射到不同的单元，但由于数组单元有限，关键字的范围可能超过数组单元。
当两个关键字散列到同一个值的时候，就是散列冲突。

e.g: 假设有一个大小为7的表，要将13， 18， 19， 50 散列到表中  

table：  
idx: 0 | 1 | 2 | 3 | 4 | 5 | 6 |  

val:   |50 |   |   |18 |19 |13 |  

table[i%7]= i, 可以快速从散列表中查找数据

如果再插入20到这个表中，则会有散列冲突。
解决散列冲突的方法通常有：  
1.拉链法 (将同一个值的关键字保存在同一个表中，另外分配新的单元来存储散列到同一个位置的数据，但是在查找的时候，
除了根据计算出的散列值找到对应的位置外，还需要在链表中进行搜索，查找效率低）   

2.开放定址法 （如果发生冲突，就选择另一个可用的位置，例如线性探测法）20%7=6 （6+1）%6=1  

3.再散列法 （建立一个是原来两倍大小的散列表，将原来表中的关键字重新散列到新表中）

In [3]:
# 用散列表来存储模式串的字符和下标 
# 便于查找字符串在模式串中的位置
def generatebc(pattern):
    bc = [-1] * 265 
    n = len(pattern)
    for i in range(n):
        index = ord(pattern[i])
        bc[index] = i
    return bc

查找主串中「不匹配的字符」（坏字符）在模式串中的位置  
1. 主串中坏字符的位置对应在模式串中的位置 main_str index（s_i)  
2. 主串的坏字符在模式串的位置 pattern index（x_i)  

模式串移动的距离为（s_i-x_i)

In [4]:
def findBadChar(substr, pattern, bc):
    n = len(substr) - 1
    j = -1 # s_i
    k = -1 # x_i
    badchar = ''
    for i in range(n, -1, -1):
        if substr[i] != pattern[i]:
            j = i
            badchar = substr[i]
            break
    if j > 0: 
        k = bc[ord(badchar)]
    return j, k
   
    
substr = 'babc'
pattern = 'abce'
bc = generatebc(pattern)
j, k = findBadChar(substr, pattern, bc)
print(j) 
print(k)

3
2


坏字符原则
1. 如果模式串中没有出现「坏字符」，模式串整体移动 len(pattern)的长度
2. 如果模式串中出现「坏字符」，模式串移动，使得模式串最靠右的字符与坏字符的位置相对

In [5]:
main_str = 'bfabce'
pattern = 'abce'

def move(main_str, pattern):
    n = len(main_str)
    m = len(pattern)
    if n == 0 or m == 0 or n < m:
        return -1
    for i in range(0, n-m+1):
        substr = main_str[i:i+m]  # last: main_str[n-m:n] 
        bc = generatebc(pattern)
        txt_idx, pat_idx = findBadChar(substr, pattern, bc)
        if txt_idx != -1 and pat_idx == -1: # 模式串中没有「坏字符」
            step = m
            if (i+2*m) <= n:
                substr = main_str[i+m:i+2*m]
                if substr == pattern:
                    print("Pattern occur at shift = {}".format(i+step))
                    print("Pattern stop at shift = {}".format(i+m+step-1))
                    return True
        if txt_idx != -1 and pat_idx != -1: # # 模式串中出现「坏字符」
            step = txt_idx - pat_idx
            if (i+m+step) <= n:
                substr = main_str[i+step:i+m+step]
                if substr == pattern:
                    print("Pattern occur at shift = {}".format(i+step))
                    print("Pattern stop at shift = {}".format(i+m+step-1))
                    return True
    return False
                
print(move(main_str, pattern))            
        


Pattern occur at shift = 2
Pattern stop at shift = 5
True


In [6]:
main_str2 = 'aaaa'
pattern2 = 'ba'
print(move(main_str2, pattern2))

False


好后缀规则  
1. 找到好后缀的所有后缀子串  
2. 找出模式串的所有前缀子串  
3. 找到好后缀中最长的能和模式串的前缀子串匹配的后缀子串  

note: 好后缀的后缀子串，本身也是模式串的后缀子串，利用这个特点在模式串中找到对应的字符


In [7]:
# generateGS 判断模式串中是否存在和好后缀相同的字符串
# suffix[k] = n ，子串长度为k，它对应的前缀的起始位置为i，即[i, i+1, ..., i+k-1]
# prefix, Boolean, 记录模式串是否存在和后缀相同的字符串
def generateGS(pattern, substr):
    n = len(pattern)
    suffix = [-1 for i in range(n)]
    prefix = [False for i in range(n)]
    for i in range(n):
        j = i
        k = 0 # 子串的长度
        while (j >=0) and (pattern[j] == substr[n-1-k]):
            k = k+1
            suffix[k] = j 
            j = j-1
        if j == -1:
            prefix[k] = True
    return suffix, prefix

# pattern[0] == main_str[3]
# k = 1
# suffix[1] = 0 
# prefix[1] = True



main_str3 = 'ecab'
pattern = 'bcab'
suffix, prefix = generateGS(pattern, main_str3)
print(suffix)
print(prefix)
        

[-1, 3, 2, 1]
[False, True, False, False]


***kmp算法***

prefix 始终包括第一个字符，但不包括字符串的最后一个字符。  
suffix 始终包括最后一个字符，但不包括字符串的第一个字符。   

next[j]=k 代表j之前的字符串中有最大长度为k的相同的前缀后缀。
j = next[j] 相当于模式串右移了 j-next[j]的距离


In [16]:
def KmpSearch(txt, pattern):
    n = len(txt)
    i = 0 # txt_index
    m = len(pattern)
    j = 0 # pattern_index
    _next = getNext(pattern) # 基于next数组进行匹配
    while i < n and j < m:
        if j == -1 or txt[i] == pattern[j]:
            i = i+1
            j = j+1
        elif j != -1 and txt[i] != pattern[j]:
            j = _next[j] # 递归
    
    # 匹配成功
    if j == m:
        return (i - j)
    # 匹配失败
    return -1
        
# [P_0, P_1, ..., P_{K-1}] P_{K}, .... [P_{j-k}, P_{j-k+1}, ..., P_{j-1}] P_{j}  
# P[k] == P[j] : next[j+1] = k+1 注意边界条件 j+1 < m 
# P[k] != P[j] : k = next[k] 
def getNext(pattern):
    m = len(pattern)
    _next = [-1]*m
    k = -1 # prefix index
    j = 0 # suffix index
    while j < (m-1):
        if (k == -1 or pattern[k] == pattern[j]):
            k = k+1
            j = j+1
            _next[j] = k
        else:
            k = _next[k]
    return _next
            
txt = 'cdfabde' 
pattern = 'abd'
find = KmpSearch(txt, pattern)
print(find)

pattern2 = 'DABCDABDE'
print(getNext(pattern2))
    

3
[-1, 0, 0, 0, 0, 1, 2, 3, 1]
